In [2]:
from cirq_sic import *
from cirq.contrib.svg import SVGCircuit

import numpy as np
np.set_printoptions(precision=3, suppress=True)

import matplotlib.pyplot as plt

In [3]:
processor_id = "willow_pink"
globals().update(get_device_data(processor_id))

In [23]:
n = 2
d = 2**n
phi = load_sic_fiducial(d)

q = cirq.GridQubit.rect(2, n, top=5, left=1)
A, B = q[:n], q[n:]
prepare_ansatz = ansatz_circuit(phi)
circuit = cirq.Circuit((prepare_ansatz(A),\
                        simple_wh_povm(A, B, prepare_ansatz, measure=True)))
optimized_circuit = process_circuit(circuit, connectivity_graph, gateset, A+B)
device.validate_circuit(optimized_circuit)

In [24]:
# Exact simulation
N = 10000
s = cirq.Simulator()
samples = s.run(optimized_circuit, repetitions=N)
counts = samples.histogram(key="result")
exact_freqs =  np.array([v for k, v in sorted(counts.items())])/N
exact_freqs

array([0.25 , 0.051, 0.052, 0.052, 0.053, 0.048, 0.052, 0.051, 0.046,
       0.048, 0.049, 0.049, 0.048, 0.054, 0.05 , 0.048])

In [25]:
# Noisy simulation
N = 10000
samples = sampler.run(optimized_circuit, repetitions=N)
counts = samples.histogram(key="result")
noisy_freqs = np.array([v for k, v in sorted(counts.items())])/N; noisy_freqs

array([0.234, 0.059, 0.052, 0.036, 0.049, 0.051, 0.063, 0.058, 0.049,
       0.059, 0.062, 0.053, 0.048, 0.049, 0.045, 0.033])

In [29]:
get_gate_counts(optimized_circuit)

--- Gate Counts (by type) ---
PhasedXPowGate: 34
CZPowGate: 26
ZPowGate: 2
PhasedXZGate: 19
MeasurementGate: 1


In [30]:
 len(optimized_circuit.moments)

43